# Análise de Dados

## Market Basket Analysis


A análise de cesta de mercado é amplamente utilizada no setor de varejo, onde os dados de vendas são processados e analisados para conseguir entender os padrões de compra dos clientes e assim, alcançar as melhores estratégias de venda.

Para o presente estudo, será utilizado um conjunto de dados público, “The Instacart Online Grocery Shopping Dataset 2017” (https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) com mais de 3 milhoões de pedidos de mais de 200 mil usuários do instacart. Será utilizado o algoritmo Apriori, que consiste na mineração de conjuntos de itens frequentes, ou seja, ele agrupa os itens que mais são comprados juntos pelos clientes, indicando suas tendências e correlações.

Dataset: 
<br>
https://www.kaggle.com/c/instacart-market-basket-analysis/data

![title](Instacart_modelagem.png)

##### Aisles (Corredores):
* aisle - corredor do produto
* aisle_id - id do corredor

##### departments (departamentos):
* departments - departamento que o produto pertence
* departments_id - id do departamento

##### products (produtos):
* product_name - nome do produto
* product_id - id do produto
* aisle_id - id do corredor onde o respectivo produto se encontra
* departments_id - id do departamento ao qual o respectivo produto pertence

##### orders (pedidos):
* order_id - id do pedido
* user_id - id do cliente que fez o pedido
* eval_set - indica a qual conjunto de dados a respectiva linha pertence, se ao dataset de pedidos anteriores (order_products_prior) ou ao dataset de treino (order_products_train)
* order_number - número do pedido
* order_dow - dia da semana em que o pedido foi feito
* order_hour_of_day - hora do dia em que o pedido foi realizado
* days_since_prior_order - dias desde o último pedido do respectivo cliente

##### order_products__* (tabela dimensão entre ps pedidos e os produtos):
* order_id - id do pedido
* product_id - id do produto
* add_to_cart_order - ordem em que o produto foi adicionado ao carrinho
* reordered - indica se o produto foi reordenado ou não, dentro do carrinho

In [1]:
# Bibliotecas utilizadas no projeto
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
import gc
from platform import python_version

In [2]:
# Ignore possíveis avisos de futuras atualizações de bibliotecas
warnings.filterwarnings('ignore')

In [3]:
# versão do python utilizada no projeto
print("Versão do python: ",python_version())

Versão do python:  3.11.5


In [4]:
# Carregando conjunto de dados
tabelas = {'corredores':pd.read_csv(r"datasets\aisles.csv"), 
           'departamentos':pd.read_csv(r"datasets\departments.csv"), 
           'pedidos':pd.read_csv(r"datasets\orders.csv"), 
           'produtos':pd.read_csv(r"datasets\products.csv"), 
           'pedidos_anteriores':pd.read_csv(r"datasets\order_products__prior.csv"), 
           'pedidos_treino':pd.read_csv(r"datasets\order_products__train.csv")}

In [5]:
# Quantidade de registros em cada tabela
contagem_registros = np.array([])
contagem_registros = pd.DataFrame([np.append(contagem_registros,[tabela[0],len(tabela[1])]) for tabela in tabelas.items()], columns = ['Tabela', 'Quantidade de registros'])
contagem_registros

,Tabela,Quantidade de registros
0,corredores,134
1,departamentos,21
2,pedidos,3421083
3,produtos,49688
4,pedidos_anteriores,32434489
5,pedidos_treino,1384617


In [6]:
# Verificando se existem dados faltantes em alguma das tabelas
dados_nulos = [{tabela[0]: tabela[1].isna().sum()} for tabela in tabelas.items() if tabela[1].isna().sum().any()!=0]
dados_nulos

[{'pedidos': order_id                       0
  user_id                        0
  eval_set                       0
  order_number                   0
  order_dow                      0
  order_hour_of_day              0
  days_since_prior_order    206209
  dtype: int64}]

# Algoritmo Apriori

O **algoritmo Apriori** é usado para minerar **conjuntos de itens frequentes** e **regras de associação** em bancos de dados. Ele é amplamente aplicado em análise de cestas de compras, onde identifica padrões de compra com base nos itens frequentemente adquiridos juntos.

## Métricas Importantes

### 1. Suporte (Support)
O suporte mede a frequência com que um conjunto de itens aparece no banco de dados. É calculado da seguinte forma:

$$ \text{Suporte} = \frac{\text{freq}(X, Y)}{N} $$

onde:
- $\text{freq}(X, Y)$ é o número de ocorrências do conjunto de itens $X$ e $Y$.
- $N$ é o tamanho total do banco de dados.

### 2. Confiança (Confiança)
A confiança mede a probabilidade de que um item $Y$ seja comprado dado que o item $X$ foi comprado. A regra de associação é representada como $X \Rightarrow Y$. A confiança é calculada assim:

$$ \text{Confiança} = \frac{\text{freq}(X)}{\text{freq}(Y)} $$

### 3. Elevação (Lift)
A elevação compara a probabilidade de que os itens $X$ e $Y$ sejam comprados juntos com a probabilidade esperada se eles fossem independentes. É dado por:

$$ \text{Elevação} = \frac{\text{Suporte}}{\text{Supp}(Y)} $$

onde:
- $\text{Supp}(Y)$ é o suporte do item $Y$.

## Exemplo

Considere o seguinte conjunto de cestas de compras:

| Cesta | Itens |
| --- | --- |
| A | {A, B, C} |
| B | {A, C, D} |
| C | {B} |
| D | {C, D} |
| E | {A, D, E} |

Com base nesses dados, podemos derivar as seguintes regras de associação:

1. $A \Rightarrow D$:
   - Suporte: $\frac{2}{5}$
   - Confiança: $\frac{2}{3}$
   - Elevação: $\frac{10}{9}$

2. $C \Rightarrow A$:
   - Suporte: $\frac{2}{5}$
   - Confiança: $\frac{2}{4}$
   - Elevação: $\frac{5}{6}$

3. $A = D$:
   - Suporte: $\frac{1}{5}$
   - Confiança: $\frac{1}{3}$
   - Elevação: $\frac{5}{9}$

Lembre-se de que esses valores podem variar dependendo do conjunto de dados e dos parâmetros definidos para o algoritmo Apriori.


#### Processamento de dados em lote:
Por se tratar de um conjunto de dados com mais de 30 milhões de linhas, uma solução para evitar sobrecarga é processar os dados em lotes, definindo o tamanho de cada lote (chunk_size) e o máximo de linhas do conjunto de dados que se deseja processar (max_rows).

In [7]:
file_path = r"datasets\order_products__prior.csv"
chunk_size = 100000
min_support = 0.01
min_confidence = 0.2
max_rows = 3000000
all_frequent_itemsets = pd.DataFrame()
total_transactions = 0
count = 0

In [8]:
# Função para processar cada lote
def process_chunk(chunk):
    # Codificação dos dados
    data_encoded = pd.get_dummies(chunk.sort_values(['order_id', 'add_to_cart_order'], ascending=[True, True])[['order_id', 'product_id']].astype('int32'), columns=['product_id'])
    data_encoded.columns = data_encoded.columns.str.replace("product_id_", "")
    #display(data_encoded)
    
    # Criação do DataFrame basket
    basket = data_encoded.groupby('order_id').sum()
    basket[basket > 0] = True
    basket[basket == 0] = False

    # Aplicação do algoritmo Apriori
    frequent_itemsets = apriori(basket, min_support=min_support, use_colnames=True)
    
    return frequent_itemsets

In [9]:
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    if total_transactions >= max_rows:
        break
    
    # Limitar o número de linhas do chunk caso exceda o máximo permitido
    if total_transactions + len(chunk) > max_rows:
        chunk = chunk.iloc[:max_rows - total_transactions]
    
    # Atualizar a contagem total de transações
    total_transactions += len(chunk)
        
    count += 1
    
    # Processar o chunk e obter os frequent itemsets
    frequent_itemsets = process_chunk(chunk)
    
    # Acumular os resultados
    all_frequent_itemsets = pd.concat([all_frequent_itemsets, frequent_itemsets])
    
    # Limpar a memória
    del chunk
    gc.collect()


Após o processamento em lote, vamos juntar os resultados de cada um deles pela média. É importante notar que foi considerado um peso para cada agregação feita, para evitar que correlações que aparecessem em apenas 1 dos lotes ficassem entre os melhores classificados.

In [10]:
all_frequent_itemsets_mean = all_frequent_itemsets.groupby('itemsets').agg(
    support=('support', 'mean'),
    count=('support', 'size')
).reset_index()

all_frequent_itemsets_mean['peso'] = all_frequent_itemsets_mean['count']/count
all_frequent_itemsets_mean = all_frequent_itemsets_mean[all_frequent_itemsets_mean['peso'] >= 0.6]
all_frequent_itemsets_mean = all_frequent_itemsets_mean[['itemsets', 'support']]
all_frequent_itemsets_mean

,itemsets,support
0,(196),0.011472
1,(49683),0.030211
2,"(21137, 13176)",0.019023
3,"(21903, 13176)",0.01554
4,"(27966, 13176)",0.012381
...,...,...
153,(27845),0.042784
154,(27966),0.042095
155,(28199),0.012643
156,(28204),0.027866


In [11]:
rules = association_rules(all_frequent_itemsets_mean, metric="confidence", min_threshold=min_confidence)
rules = rules.sort_values(['confidence', 'lift'], ascending=[False, False])

# Exibir as regras
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
7,(28204),(24852),0.027866,0.147783,0.010967,0.393581,2.663235,0.006849,1.405327,0.642418
8,(47766),(24852),0.054334,0.147783,0.016412,0.302049,2.043866,0.008382,1.221026,0.540076
2,(27966),(13176),0.042095,0.117791,0.012381,0.294124,2.497005,0.007423,1.249808,0.625866
4,(16797),(24852),0.044694,0.147783,0.013116,0.293469,1.985811,0.006511,1.206199,0.519653
3,(47209),(13176),0.065966,0.117791,0.019227,0.291466,2.474440,0.011457,1.245119,0.637951
10,(47626),(24852),0.047724,0.147783,0.012837,0.268992,1.820182,0.005785,1.165811,0.473187
9,(27966),(21137),0.042095,0.082083,0.010822,0.257087,3.132016,0.007367,1.235563,0.710630
0,(21137),(13176),0.082083,0.117791,0.019023,0.231755,1.967517,0.009355,1.148344,0.535719
6,(21903),(24852),0.075323,0.147783,0.016230,0.215478,1.458070,0.005099,1.086288,0.339753
5,(21137),(24852),0.082083,0.147783,0.017500,0.213203,1.442677,0.005370,1.083147,0.334283


In [12]:
id_to_name = dict(zip(tabelas['produtos']['product_id'], tabelas['produtos']['product_name']))

In [13]:
def map_ids_to_names(itemset):
    return frozenset([id_to_name[int(item)] for item in itemset])

In [14]:
rules['antecedents'] = rules['antecedents'].apply(map_ids_to_names)
rules['consequents'] = rules['consequents'].apply(map_ids_to_names)

Após a execução do algoritmo, obteve-se as principais correlações listadas abaixo, onde pessoas que compram maçã fuji orgânica tendem a comprar banana também, por exemplo.

In [15]:
rules.sort_values(by = 'confidence', ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
7,(Organic Fuji Apple),(Banana),0.027866,0.147783,0.010967,0.393581,2.663235,0.006849,1.405327,0.642418
8,(Organic Avocado),(Banana),0.054334,0.147783,0.016412,0.302049,2.043866,0.008382,1.221026,0.540076
2,(Organic Raspberries),(Bag of Organic Bananas),0.042095,0.117791,0.012381,0.294124,2.497005,0.007423,1.249808,0.625866
4,(Strawberries),(Banana),0.044694,0.147783,0.013116,0.293469,1.985811,0.006511,1.206199,0.519653
3,(Organic Hass Avocado),(Bag of Organic Bananas),0.065966,0.117791,0.019227,0.291466,2.474440,0.011457,1.245119,0.637951
10,(Large Lemon),(Banana),0.047724,0.147783,0.012837,0.268992,1.820182,0.005785,1.165811,0.473187
9,(Organic Raspberries),(Organic Strawberries),0.042095,0.082083,0.010822,0.257087,3.132016,0.007367,1.235563,0.710630
0,(Organic Strawberries),(Bag of Organic Bananas),0.082083,0.117791,0.019023,0.231755,1.967517,0.009355,1.148344,0.535719
6,(Organic Baby Spinach),(Banana),0.075323,0.147783,0.016230,0.215478,1.458070,0.005099,1.086288,0.339753
5,(Organic Strawberries),(Banana),0.082083,0.147783,0.017500,0.213203,1.442677,0.005370,1.083147,0.334283
